In [12]:
!apt-get -qq install -y python3-libtorrent libtorrent-rasterbar-dev #for utilities used inside
!apt-get -qq install -y zip
import tensorflow as tf
import libtorrent as lt
from google.colab import drive
from google.colab import files
import time
drive.mount('/content/drive')  #mounts your Drive 
start_time = time.time()
#torrenting 
ses= lt.session()   #get session handle
ses.listen_on(6896,6884)

tparams = {"save_path":".",
          "storage_mode":lt.storage_mode_t(2),
          "paused":False,
          "auto_managed":True,
          "duplicate_is_error":False} #Torrent physical Parameters
link = "magnet:?xt=urn:btih:38ED63858AD5A5A5214C6D7250256C1333597415&dn=Angrezi+Medium+%282020%29+\
Hindi+HQ+720p+HDCAM+x264+MP3+Line+Audios+-+Shadow&tr=udp%3A%2F%2Ftracker.coppersurfer.tk%3A6969%2\
Fannounce&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2Fretracker.lanta-n\
et.ru%3A2710%2Fannounce&tr=udp%3A%2F%2Ftracker.cyberia.is%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker\
.leechers-paradise.org%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.tiny-vps.com%3A6969%2Fannounce&tr=\
udp%3A%2F%2F9.rarbg.com%3A2710%2Fannounce&tr=udp%3A%2F%2F9.rarbg.me%3A2730%2Fannounce&tr=udp%3A%2F\
%2Ftracker.pirateparty.gr%3A6969%2Fannounce&tr=http%3A%2F%2Ftracker.files.fm%3A6969%2Fannounce&tr=h\
ttp%3A%2F%2Ftracker810.xyz%3A11450%2Fannounce&tr=udp%3A%2F%2Ftracker.skynetcloud.site%3A6969%2Fanno\
unce&tr=udp%3A%2F%2Ftracker.zer0day.to%3A1337%2Fannounce&tr=udp%3A%2F%2Ftracker.leechers-paradise.o\
rg%3A6969%2Fannounce&tr=udp%3A%2F%2Fcoppersurfer.tk%3A6969%2Fannounce"  # Magnet link of torrent
tor_handle = lt.add_magnet_uri(ses, link, tparams) #initializing torrent with parameters
ses.start_dht()
print("Fetching Metadata...")    #torrent activity starts
while(not tor_handle.has_metadata()):
    time.sleep(1)
print("Starting the Torrent...")
while(tor_handle.status().state != lt.torrent_status.seeding ):   #download progress 
    s = tor_handle.status()
    tor_status = ['queued','checking','fetching metadata',\
                  'downloading','finished', 'seeding', 'allocating']
    print("Progress:",s.progress*100, \
          "Down_speed:",s.download_rate/1000 ,"UP_Speed", s.upload_rate/1000,"Peers:", s.num_peers,"Status:",tor_status[s.state])
    time.sleep(5)
#/torrenting
print("---Torrent Downloaded in  %s seconds ---" % (time.time() - start_time))
#! zip -r filename.zip "Angrezi Medium (2020) Hindi 720p HDCAM x264 MP3 HQ Line Audios - Shadow"
#! rm -rf 'Angrezi Medium (2020) Hindi 720p HDCAM x264 MP3 HQ Line Audios - Shadow'
print("came here !")
#!ls -a
!mv "Angrezi Medium (2020) Hindi 720p HDCAM x264 MP3 HQ Line Audios - Shadow" "/content/drive/My Drive/" #need to update so name is not hard coded
#!mv filename.zip "/content/drive/My Drive/Angrezimedium.zip"
print("--- %s seconds ---" % (time.time() - start_time))


print("Done beero !")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fetching Metadata...
Starting the Torrent...
Progress: 0.0 Down_speed: 2.683 UP_Speed 2.17 Peers: 27 Status: downloading
Progress: 0.029799999902024865 Down_speed: 872.86 UP_Speed 38.685 Peers: 30 Status: downloading
Progress: 2.08320003002882 Down_speed: 3305.903 UP_Speed 124.013 Peers: 27 Status: downloading
Progress: 3.0691999942064285 Down_speed: 2852.848 UP_Speed 100.429 Peers: 36 Status: downloading
Progress: 5.456500127911568 Down_speed: 5007.841 UP_Speed 170.284 Peers: 42 Status: downloading
Progress: 7.7573999762535095 Down_speed: 5790.14 UP_Speed 195.707 Peers: 50 Status: downloading
Progress: 10.693199932575226 Down_speed: 7581.243 UP_Speed 260.712 Peers: 57 Status: downloading
Progress: 14.288200438022614 Down_speed: 9403.443 UP_Speed 329.394 Peers: 60 Status: downloading
Progress: 17.428100109100342 Down_speed: 8242.059 UP_Speed 295.012 Peers: 61